<a href="https://colab.research.google.com/github/guscldns/TestProject/blob/main/%EC%9E%A5%EA%B8%B0/%ED%8C%8C%EC%9D%B4%ED%94%84%EB%9D%BC%EC%9D%B8py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install keybert
!pip install kiwipiepy
!pip install google-cloud-vision
!pip install PyMuPDF
!pip install transformers

In [37]:
import numpy as np
import fitz
import zipfile
import cv2
import io
import os
import torch
from keybert import KeyBERT
from kiwipiepy import Kiwi
from transformers import BertModel
from PIL import Image
from google.cloud import vision
from transformers import PreTrainedTokenizerFast
from transformers.models.bart import BartForConditionalGeneration

import torch
from transformers import PreTrainedTokenizerFast
from transformers.models.bart import BartForConditionalGeneration, BartForCausalLM
# 모델과 토크나이저 로드
# model = BartForConditionalGeneration.from_pretrained('/content/kobart_summary')
summary_model = BartForConditionalGeneration.from_pretrained('/content/drive/MyDrive/KoBART-summarization/kobart_summary')
summary_tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-base-v1')


### 키워드 명사로만 제공
from kiwipiepy import Kiwi
from transformers import BertModel
from keybert import KeyBERT

# 텍스트 전문 : full_text
# Kiwi 형태소 분석기 초기화
kiwi = Kiwi()

# BERT 모델 로드
keywords_model = BertModel.from_pretrained('skt/kobert-base-v1')

# KeyBERT 모델 초기화
kw_model = KeyBERT(keywords_model)

# 파일 저장할 폴더 생성
# os.mkdir("/content/upload_file")
# os.mkdir("/content/image_file")

# PDF 라면 이미지로 변경
def pdf_to_png(files):
    path = f"/content/upload_file/{files}"
    doc = fitz.open(path)
    for i, page in enumerate(doc):
        img = page.get_pixmap()
        img.save(f"/content/image_file/{i}.png")

# OCR 함수
def detect_paragraphs(image_path):
    from google.cloud import vision
    # API키 가져오기
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = r"/content/drive/MyDrive/api-project-397750607032-5ddc025931cd.json"

    # API 가져오기
    client = vision.ImageAnnotatorClient()

    # 주석을 추가할 이미지 파일 이름
    file_name = os.path.abspath(image_path)

    # 이미지 로드
    with io.open(file_name, 'rb') as image_file:
        content = image_file.read()

    image = vision.Image(content=content)

    # 이미지 OCR
    response = client.document_text_detection(image=image)

    # 이미지 OCR 텍스트 전문
    full_text = response.full_text_annotation.text

    # 이미지 OCR 후 결과 (bbox, word 등)
    pages = response.full_text_annotation.pages

    # 텍스트 주석
    # 참고 : https://cloud.google.com/dotnet/docs/reference/Google.Cloud.Vision.V1/latest/Google.Cloud.Vision.V1.TextAnnotation.Types.DetectedBreak.Types.BreakType
    # 참고 : https://googleapis.github.io/googleapis/java/grpc-google-cloud-vision-v1/0.1.5/apidocs/com/google/cloud/vision/v1/TextAnnotation.DetectedBreak.BreakType.html
    breaks = vision.TextAnnotation.DetectedBreak.BreakType

    paragraphs = []
    lines = []

    for page in pages:
        for block in page.blocks:
            for paragraph in block.paragraphs:
                para = "★문단시작★"
                line = ""
                for word in paragraph.words:
                    for symbol in word.symbols:
                        line += symbol.text
                        # breaks.SPACE : 공백
                        if symbol.property.detected_break.type == breaks.SPACE:
                            line += ' '
                        # breaks.EOL_SURE_SPACE : 줄 바꿈
                        if symbol.property.detected_break.type == breaks.EOL_SURE_SPACE:
                            line += ' '
                            lines.append(line)
                            para += line
                            line = ''
                        # breaks.LINE_BREAK : 단락을 끝내는 줄바꿈
                        if symbol.property.detected_break.type == breaks.LINE_BREAK:
                            lines.append(line)
                            para += line
                            line = ''
                paragraphs.append(para)

    return full_text, paragraphs

def summarize_paragraphs(paragraphs):

    # 문단 리스트를 512 토큰으로 나누기
    # 입력문장 리스트 : paragraphs

    # 모델 입력 데이터 생성
    input_data = []

    # 요약 데이터 생성
    summary =[]

    # 현재까지의 토큰 수 초기화
    current_token_count = 0

    for paragraph in paragraphs:
        # '★문단시작★'를 제거하고 문단을 토큰화
        paragraph_tokens = summary_tokenizer(paragraph.replace('★문단시작★', ''), return_tensors='pt', add_special_tokens=True).input_ids

        # 현재 문단을 추가해도 512 토큰을 넘지 않으면 추가
        if current_token_count + len(paragraph_tokens[0]) <= 512:
            if not input_data:
                input_data.append(paragraph_tokens)
                current_token_count += len(paragraph_tokens[0])
            else:
                # 이미 문단이 추가되어 있는 경우 이어서 추가
                input_data[-1] = torch.cat((input_data[-1], paragraph_tokens), dim=-1)
                current_token_count += len(paragraph_tokens[0])
        else:
            # 256 토큰을 넘어가면 새로운 입력으로 시작
            input_data.append(paragraph_tokens)
            current_token_count = len(paragraph_tokens[0])

    for inp in input_data:
        output = summary_model.generate(inp, eos_token_id=1, max_length=512, num_beams=4)
        t_output = summary_tokenizer.decode(output[0], skip_special_tokens=True)
        summary.append(t_output)

    return summary

def fulltext_keywords(full_text):

    kiwi.analyze(full_text)

    # 키워드 추출
    keywords = kw_model.extract_keywords(full_text, keyphrase_ngram_range=(1, 1), stop_words=None, top_n=10)

    # 명사 키워드 추출 함수 정의
    def extract_nouns(keyword_list):
        noun_keywords = []
        keywordscore = []
        for keyword, score in keywords:
            # 형태소 분석 수행
            tokens = kiwi.analyze(keyword)
            # print(tokens)
            for tk in tokens[0][0]:
                # print(tk)
                if tk.tag == "NNG":
                    if tk.form not in noun_keywords:
                        noun_keywords.append(tk.form)
                        keywordscore.append((tk.form,score))

        return noun_keywords, keywordscore
    noun_keywords, keywordscore = extract_nouns(keywords)
    return noun_keywords, keywordscore


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [8]:
# 코렙이라서 사용, flask에서는 다른 방법 필요
%cd /content/upload_file
from google.colab import files
uploaded = files.upload()

file_names = os.listdir('/content/upload_file')
if '.ipynb_checkpoints' in file_names:
    file_names.remove('.ipynb_checkpoints')

for files in file_names:
    if '.pdf' in files[-4:]:
	    pdf_to_png(files)

summarys, keywordscores, noun_keyword = [], [], []
full_text = ""
image_pathes = os.listdir('/content/image_file')
 # PDF 10장/ 2분 7초
for image_paths in image_pathes:
    image_path = f'/content/image_file/{image_paths}'
    text_full, paragraphs = detect_paragraphs(image_path)
    full_text = text_full
    summarys.append(summarize_paragraphs(paragraphs))
    noun_keywords, keywordscore = fulltext_keywords(full_text)
    keywordscores.append(keywordscore)
    noun_keyword.append(noun_keywords)

from collections import Counter
# 키워드 리스트 하나로 합치기
word_list = sum(noun_keyword,[])
# 단어 빈도수 계산
word_counts = Counter(word_list)

# 가장 많이 중복된 단어 5개 선택
top_5_words = [word for word, count in word_counts.most_common(5)]

print(top_5_words)

/content/upload_file


Saving 현진건-운수좋은날+B3356-개벽.pdf to 현진건-운수좋은날+B3356-개벽.pdf


In [45]:
import numpy as np
import fitz
import zipfile
import cv2
import io
import os
import torch
from keybert import KeyBERT
from kiwipiepy import Kiwi
from transformers import BertModel
from PIL import Image
from google.cloud import vision
from transformers import PreTrainedTokenizerFast
from transformers.models.bart import BartForConditionalGeneration

import torch
from transformers import PreTrainedTokenizerFast
from transformers.models.bart import BartForConditionalGeneration, BartForCausalLM
# 모델과 토크나이저 로드
# model = BartForConditionalGeneration.from_pretrained('/content/kobart_summary')
summary_model = BartForConditionalGeneration.from_pretrained('/content/drive/MyDrive/KoBART-summarization/kobart_summary')
summary_tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-base-v1')


### 키워드 명사로만 제공
from kiwipiepy import Kiwi
from transformers import BertModel
from keybert import KeyBERT

# 텍스트 전문 : full_text
# Kiwi 형태소 분석기 초기화
kiwi = Kiwi()

# BERT 모델 로드
keywords_model = BertModel.from_pretrained('skt/kobert-base-v1')

# KeyBERT 모델 초기화
kw_model = KeyBERT(keywords_model)

# 파일 저장할 폴더 생성
os.mkdir("/content/upload_file")
os.mkdir("/content/image_file")

# PDF 라면 이미지로 변경
def pdf_to_png(files):
    path = f"/content/upload_file/{files}"
    doc = fitz.open(path)
    for i, page in enumerate(doc):
        img = page.get_pixmap()
        img.save(f"/content/image_file/{i}.png")

# OCR 함수
def detect_paragraphs(image_path):
    from google.cloud import vision
    # API키 가져오기
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = r"/content/drive/MyDrive/api-project-397750607032-5ddc025931cd.json"

    # API 가져오기
    client = vision.ImageAnnotatorClient()

    # 주석을 추가할 이미지 파일 이름
    file_name = os.path.abspath(image_path)

    # 이미지 로드
    with io.open(file_name, 'rb') as image_file:
        content = image_file.read()

    image = vision.Image(content=content)

    # 이미지 OCR
    response = client.document_text_detection(image=image)

    # 이미지 OCR 텍스트 전문
    full_text = response.full_text_annotation.text

    # 이미지 OCR 후 결과 (bbox, word 등)
    pages = response.full_text_annotation.pages

    # 텍스트 주석
    # 참고 : https://cloud.google.com/dotnet/docs/reference/Google.Cloud.Vision.V1/latest/Google.Cloud.Vision.V1.TextAnnotation.Types.DetectedBreak.Types.BreakType
    # 참고 : https://googleapis.github.io/googleapis/java/grpc-google-cloud-vision-v1/0.1.5/apidocs/com/google/cloud/vision/v1/TextAnnotation.DetectedBreak.BreakType.html
    breaks = vision.TextAnnotation.DetectedBreak.BreakType

    paragraphs = []
    lines = []

    for page in pages:
        for block in page.blocks:
            for paragraph in block.paragraphs:
                para = "★문단시작★"
                line = ""
                for word in paragraph.words:
                    for symbol in word.symbols:
                        line += symbol.text
                        # breaks.SPACE : 공백
                        if symbol.property.detected_break.type == breaks.SPACE:
                            line += ' '
                        # breaks.EOL_SURE_SPACE : 줄 바꿈
                        if symbol.property.detected_break.type == breaks.EOL_SURE_SPACE:
                            line += ' '
                            lines.append(line)
                            para += line
                            line = ''
                        # breaks.LINE_BREAK : 단락을 끝내는 줄바꿈
                        if symbol.property.detected_break.type == breaks.LINE_BREAK:
                            lines.append(line)
                            para += line
                            line = ''
                paragraphs.append(para)

    return full_text, paragraphs

def summarize_paragraphs(paragraphs):

    # 문단 리스트를 512 토큰으로 나누기
    # 입력문장 리스트 : paragraphs

    # 모델 입력 데이터 생성
    input_data = []

    # 요약 데이터 생성
    summary =[]

    # 현재까지의 토큰 수 초기화
    current_token_count = 0

    for paragraph in paragraphs:
        # '★문단시작★'를 제거하고 문단을 토큰화
        paragraph_tokens = summary_tokenizer(paragraph.replace('★문단시작★', ''), return_tensors='pt', add_special_tokens=True).input_ids

        # 현재 문단을 추가해도 512 토큰을 넘지 않으면 추가
        if current_token_count + len(paragraph_tokens[0]) <= 512:
            if not input_data:
                input_data.append(paragraph_tokens)
                current_token_count += len(paragraph_tokens[0])
            else:
                # 이미 문단이 추가되어 있는 경우 이어서 추가
                input_data[-1] = torch.cat((input_data[-1], paragraph_tokens), dim=-1)
                current_token_count += len(paragraph_tokens[0])
        else:
            # 256 토큰을 넘어가면 새로운 입력으로 시작
            input_data.append(paragraph_tokens)
            current_token_count = len(paragraph_tokens[0])

    for inp in input_data:
        output = summary_model.generate(inp, eos_token_id=1, max_length=512, num_beams=4)
        t_output = summary_tokenizer.decode(output[0], skip_special_tokens=True)
        summary.append(t_output)

    return summary

def fulltext_keywords(full_text):

    kiwi.analyze(full_text)

    # 키워드 추출
    keywords = kw_model.extract_keywords(full_text, keyphrase_ngram_range=(1, 1), stop_words=None, top_n=10)

    # 명사 키워드 추출 함수 정의
    def extract_nouns(keyword_list):
        noun_keywords = []
        keywordscore = []
        for keyword, score in keywords:
            # 형태소 분석 수행
            tokens = kiwi.analyze(keyword)
            # print(tokens)
            for tk in tokens[0][0]:
                # print(tk)
                if tk.tag == "NNG":
                    if tk.form not in noun_keywords:
                        noun_keywords.append(tk.form)
                        keywordscore.append((tk.form,score))

        return noun_keywords, keywordscore
    noun_keywords, keywordscore = extract_nouns(keywords)
    return noun_keywords, keywordscore

def keyword_sum(noun_keyword, n):
    from collections import Counter
    # 키워드 리스트 하나로 합치기
    word_list = sum(noun_keyword,[])
    # 단어 빈도수 계산
    word_counts = Counter(word_list)

    # 가장 많이 중복된 단어 5개 선택
    top_n_words = [word for word, count in word_counts.most_common(n)]
    return top_n_words


[['김첨지는 원망스러워 인력거를 타시라고 대어섰고제야 말로 인력거가 무거워지자 마음이 초조해 온다.', '백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 백 

In [59]:
# 코렙이라서 사용, flask에서는 다른 방법 필요
%cd /content/upload_file
from google.colab import files
uploaded = files.upload()

file_names = os.listdir('/content/upload_file')
if '.ipynb_checkpoints' in file_names:
    file_names.remove('.ipynb_checkpoints')

for files in file_names:
    if '.pdf' in files[-4:]:
	    pdf_to_png(files)

summarys, keywordscores, noun_keyword = [], [], []
full_text = ""
image_pathes = os.listdir('/content/image_file')
 # PDF 10장/ 2분 7초
for image_paths in image_pathes:
    image_path = f'/content/image_file/{image_paths}'
    text_full, paragraphs = detect_paragraphs(image_path)
    full_text = text_full
    summarys.append(summarize_paragraphs(paragraphs))
    noun_keywords, keywordscore = fulltext_keywords(full_text)
    keywordscores.append(keywordscore)
    noun_keyword.append(noun_keywords)
key_5 =  keyword_sum(noun_keyword, 5)

print(summarys, full_text, key_5)